In [38]:
import sys
import glob
import serial

def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

print(serial_ports())

['/dev/tty.E500Pro', '/dev/tty.Bluetooth-Incoming-Port']


In [43]:
import serial
import time
import struct
import numpy as np

for ports in serial_ports():
    print("READING PORTS")

    ser = serial.Serial(port=ports, baudrate=115200, timeout=.1)
    ser.write(bytes([126,1]))
    time.sleep(0.1)
    ser.write(bytes([127]))
    preamb = int.from_bytes(ser.read(4), byteorder='big')
    if preamb == 0:
        print("Wrong port number {}, continuing".format(ports))
        continue
    else:
        print("Correct port number {}, getting the configurations.".format(ports))

    configurations = np.frombuffer(ser.read(8), dtype=np.uint8, count=8)
    info_1 = {"Preamble: ": preamb, "Command code sent": configurations[0], 
                "Current DAQ configuration (board type)": configurations[1], 
                "Number of used ADC channels": configurations[2], 
                "Number of bytes in one channel of ADC": configurations[3],
                "Number of connected optical sensors": configurations[4], 
                "Number of channels of optical sensors": configurations[5],
                "Number of bytes in one optical channel": configurations[6], 
                "Sampling rate of optical sensor": configurations[7]}
    print("Configurations: ", info_1)
    ser.write(bytes([0, 6]))
    print("START ACQUISITION (0, 6)")
    preamb = int.from_bytes(ser.read(4), byteorder='big')
    start_info = np.frombuffer(ser.read(3), dtype=np.uint8, count=3)
    info_2 = {"Preamble: ": preamb, "ID": start_info[0], "Not used_1": start_info[1], "Not used_2": start_info[2]}
    print("START INFO: ", info_2)
    
    seconds = 20
    print("Starting data logger for {} seconds".format(seconds))
    start_time = time.time()
    end_time = start_time + seconds  # x seconds from now

    while time.time() < end_time:
        config_data = ser.read(8)   #ID -> [4], COUNTER -> [5], N_CHANNELS -> [6], N_BYTES -> [7]
        sensor_data = ser.read(12)
        # Convert the byte string to a NumPy array of big-endian 32-bit integers
        int_array = np.frombuffer(sensor_data, dtype=np.uint32, count=3)
        if(config_data[4] == 1):
            print(int_array)
        
ser.write(struct.pack('b', -1))
print("STOP ACQUISITION")

READING PORTS
Wrong port number /dev/tty.E500Pro, continuing
READING PORTS
Wrong port number /dev/tty.Bluetooth-Incoming-Port, continuing
READING PORTS
Correct port number /dev/tty.usbmodem5D715F7938331, getting the configurations.
Configurations:  {'Preamble: ': 2857740885, 'Command code sent': 127, 'Current DAQ configuration (board type)': 2, 'Number of used ADC channels': 1, 'Number of bytes in one channel of ADC': 2, 'Number of connected optical sensors': 2, 'Number of channels of optical sensors': 3, 'Number of bytes in one optical channel': 4, 'Sampling rate of optical sensor': 100}
START ACQUISITION (0, 6)
START INFO:  {'Preamble: ': 2857740885, 'ID': 7, 'Not used_1': 2, 'Not used_2': 0}
Starting data logger for 20 seconds
[ 682302 1677541 1019597]
[ 682841 1678730 1020248]
[ 684455 1680037 1021031]
[ 684919 1680993 1021725]
[ 686120 1682140 1021946]
[ 687052 1682887 1022340]
[ 686681 1683596 1022718]
[ 686883 1684241 1022710]
[ 686929 1684614 1022761]
[ 686676 1684997 1022518]


[ 702538 1769111 1069570]
[ 702890 1769235 1069675]
[ 703633 1769360 1069710]
[ 703390 1769404 1069785]
[ 703340 1769843 1070023]
[ 703129 1770240 1070150]
[ 703369 1770580 1070361]
[ 702945 1770823 1070631]
[ 702717 1770994 1070840]
[ 702765 1771355 1070998]
[ 703380 1771529 1071107]
[ 703582 1771609 1071339]
[ 703522 1771667 1071500]
[ 704039 1771947 1071725]
[ 704548 1772053 1071697]
[ 704641 1772104 1071883]
[ 704811 1772231 1071934]
[ 704823 1772326 1072089]
[ 705530 1772445 1072150]
[ 705536 1772583 1072214]
[ 705327 1772817 1072295]
[ 705508 1772984 1072294]
[ 706304 1773146 1072431]
[ 706577 1773153 1072600]
[ 706410 1773443 1072823]
[ 707014 1773551 1072914]
[ 707577 1773820 1073112]
[ 707640 1774054 1073300]
[ 707972 1774433 1073631]
[ 707954 1774678 1073974]
[ 708313 1775282 1074247]
[ 708710 1775624 1074602]
[ 709591 1776009 1075008]
[ 709739 1776318 1075332]
[ 709851 1776655 1075650]
[ 710285 1777002 1076037]
[ 711357 1777375 1076287]
[ 711489 1777653 1076665]
[ 712418 177

[ 795148 1849438 1121798]
[ 795349 1849779 1122020]
[ 796290 1849925 1122259]
[ 796653 1850015 1122344]
[ 797492 1850309 1122602]
[ 798053 1850610 1122644]
[ 798292 1850685 1123027]
[ 797997 1851034 1123182]
[ 798920 1851606 1123437]
[ 799859 1851884 1123689]
[ 798726 1852094 1123844]
[ 799645 1852258 1123967]
[ 800634 1852418 1124013]
[ 800635 1852523 1124072]
[ 801045 1852432 1124002]
[ 800631 1852537 1123841]
[ 800168 1852521 1123505]
[ 800244 1852533 1123348]
[ 798954 1852654 1123285]
[ 798935 1852466 1123147]
[ 799309 1852497 1123150]
[ 799030 1852809 1123242]
[ 799000 1852833 1123295]
[ 799067 1853251 1123534]
[ 799173 1853560 1123694]
[ 798496 1853545 1124078]
[ 798805 1853816 1124130]
[ 798685 1854020 1124294]
[ 799167 1854122 1124353]
[ 798793 1854123 1124113]
[ 798914 1854156 1124199]
[ 798604 1854191 1124136]
[ 798819 1854369 1124190]
[ 799238 1854435 1124304]
[ 798980 1854357 1124382]
[ 798945 1854370 1124409]
[ 799293 1854428 1124427]
[ 798862 1854682 1124542]
[ 798960 185

[ 807785 1894362 1150906]
[ 808056 1894274 1150876]
[ 808348 1894690 1151033]
[ 807880 1894694 1151161]
[ 808463 1894894 1151323]
[ 808046 1895027 1151382]
[ 808458 1895488 1151420]
[ 809329 1895698 1151661]
[ 809143 1895795 1151953]
[ 809465 1896077 1152119]
[ 809814 1896323 1152482]
[ 809841 1896582 1152651]
[ 809539 1896842 1152891]
[ 809356 1897060 1153258]
[ 808875 1897284 1153390]
[ 809840 1897603 1153523]
[ 809884 1897770 1153641]
[ 810409 1898062 1153906]
[ 810893 1898130 1154090]
[ 811057 1898041 1154159]
[ 811981 1898192 1154007]
[ 811615 1898190 1154145]
[ 811633 1898216 1153859]
[ 811506 1897767 1153684]
[ 811753 1897890 1153317]
[ 812019 1897765 1153027]
[ 810516 1897545 1152825]
[ 809786 1897351 1152730]
[ 809217 1897577 1152514]
[ 808723 1897632 1152391]
[ 808038 1897475 1152693]
[ 807965 1897767 1152540]
[ 807058 1897914 1152740]
[ 806881 1898122 1152948]
[ 806729 1898565 1153102]
[ 805991 1898607 1153175]
[ 805862 1898771 1153437]
[ 806506 1898912 1153542]
[ 806999 189

[ 824107 1929199 1175619]
[ 823938 1928952 1175552]
[ 824096 1928963 1175512]
[ 823921 1929222 1175403]
[ 824088 1929252 1175437]
[ 824874 1929319 1175665]
[ 824755 1929442 1175657]
[ 824974 1929493 1175871]
[ 825267 1929663 1175914]
[ 825917 1929852 1176016]
[ 825695 1929936 1176037]
[ 826452 1929893 1176064]
[ 826843 1930172 1176284]
[ 827452 1930147 1176353]
[ 827745 1930199 1176400]
[ 827808 1930217 1176474]
[ 828088 1930423 1176605]
[ 828304 1930676 1176631]
[ 828821 1930559 1176712]
[ 829160 1930966 1176920]
[ 829821 1931048 1176942]
[ 830178 1931205 1177161]
[ 830878 1931009 1177330]
[ 831393 1931234 1177513]
[ 831653 1931426 1177629]
[ 832175 1931767 1177856]
[ 832756 1932023 1178029]
[ 832776 1932025 1178047]
[ 833168 1932064 1177984]
[ 832815 1931736 1177933]
[ 832930 1931534 1177558]
[ 832558 1931449 1177411]
[ 832316 1931267 1176945]
[ 831072 1931057 1176513]
[ 830642 1930898 1176330]
[ 829809 1930579 1176231]
[ 830081 1930785 1176078]
[ 830031 1930800 1175886]
[ 828881 193

[ 867824 1959670 1199917]
[ 867975 1959699 1199796]
[ 868230 1959580 1200008]
[ 867789 1959779 1200042]
[ 867407 1959819 1200045]
[ 867843 1959813 1200075]
[ 868080 1960154 1200152]
[ 867642 1960271 1200042]
[ 867908 1960187 1200280]
[ 868025 1960327 1200344]
[ 868638 1960212 1200425]
[ 868718 1960539 1200576]
[ 868544 1960910 1200901]
[ 867981 1961224 1201131]
[ 869014 1961349 1201126]
[ 869090 1961190 1201366]
[ 869937 1961356 1201516]
[ 870233 1961675 1201721]
[ 870226 1961858 1201828]
[ 869969 1961771 1201808]
[ 869416 1961821 1202005]
[ 869841 1961884 1201939]
[ 870720 1961819 1201991]
[ 870848 1961901 1201892]
[ 870683 1961957 1201976]
[ 871489 1962044 1201986]
[ 871442 1962112 1202268]
[ 871580 1961895 1202305]
[ 871605 1962147 1202341]
[ 871905 1962252 1202366]
[ 871852 1962241 1202271]
[ 871798 1962096 1202065]
[ 870915 1961831 1201873]
[ 870953 1961972 1201612]
[ 870508 1961722 1201387]
[ 869245 1961605 1201115]
[ 868237 1961441 1200969]
[ 867232 1961671 1200644]
[ 865843 196

[ 828082 1974238 1206478]
[ 827076 1974293 1206572]
[ 827087 1974046 1206606]
[ 827570 1974327 1206570]
[ 827354 1974158 1206645]
[ 827965 1974355 1206804]
[ 827850 1974186 1206850]
[ 828174 1974380 1206952]
[ 828175 1974484 1206893]
[ 828142 1974546 1206984]
[ 828220 1974698 1207125]
[ 827927 1974772 1207050]
[ 828359 1974616 1206999]
STOP ACQUISITION


In [ ]:
def preambula_check(ser):
    preambula_status = 0
    preambula_count = 0
    while not preambula_status:
        byte = int.from_bytes(ser.read(1), byteorder='big')
        # print("b: ", byte)
        match preambula_count:
            case 0:
                preambula_count = 1 if byte == 0xAA else 0
            case 1:
                preambula_count = 2 if byte == 0x55 else 0
            case 2:
                preambula_count = 3 if byte == 0xAA else 0
            case 3:

In [ ]:
my_list = [[],[]]
my_list[0].append([ 392416 1155796  785500])
my_list[1].append([ 406528 1092416  707846])
my_list[0].append([ 431203 1127930  730230])
my_list[1].append([ 406528 1092416  707846])
my_list[0].append([ 431203 1127930  730230])


In [ ]:
%matplotlib inline

import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

# Create figure for plotting
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
xs = []
ys = []

# Initialize communication with TMP102

# This function is called periodically from FuncAnimation
def animate(i, xs, ys):
    global ani
    # Read temperature (Celsius) from TMP102
    temp_c = np.random.randint(10, 30)

    # Add x and y to lists
    xs.append(dt.datetime.now().strftime('%H:%M:%S.%f'))
    ys.append(temp_c)

    # Limit x and y lists to 20 items
    xs = xs[-20:]
    ys = ys[-20:]

    # Draw x and y lists
    ax.clear()
    ax.plot(xs, ys)

    # Format plot
    plt.xticks(rotation=45, ha='right')
    plt.subplots_adjust(bottom=0.30)
    plt.title('TMP102 Temperature over Time')
    plt.ylabel('Temperature (deg C)')

# Set up plot to call animate() function periodically
ani = animation.FuncAnimation(fig, animate, fargs=(xs, ys), interval=1000)
plt.show()
